# WiDS

## Data Loading

__Importing the libraries__

In [71]:
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    FunctionTransformer,
    PolynomialFeatures
)

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from sklearn.feature_selection import RFECV

from sklearn.model_selection import (
    cross_val_score,
    cross_validate,
    train_test_split,
    RandomizedSearchCV
)

from sklearn.linear_model import Ridge, Lasso

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from catboost import CatBoostRegressor
from lightgbm.sklearn import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.ensemble import StackingRegressor

from sklearn.metrics import make_scorer
# from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve

In [72]:
import numpy as np
import pandas as pd
import time
from collections import defaultdict

from torchvision import transforms, datasets, utils
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split, Dataset
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
import torch.utils.data as data
from torch import nn

In [73]:
# train_path = "../input/wids2022/train.csv"
# test_path = "../input/wids2022/test.csv"
train_path = "data/train.csv"
test_path = "data/test.csv"

In [359]:
class Dataprep(Dataset):

    def __init__(self, train_path, test_path, size=0.1):
        df = pd.read_csv(train_path)
        test_df = pd.read_csv(test_path)

        TARGET_COLUMN = "site_eui"

        train_df, val_df = train_test_split(
            df,
            test_size=0.2,
            random_state=123
        )

        self.X_train, self.y_train = train_df.drop(columns=[TARGET_COLUMN]), train_df[TARGET_COLUMN]
        self.X_val, self.y_val = val_df.drop(columns=[TARGET_COLUMN]), val_df[TARGET_COLUMN]
        self.X_test = test_df

    def engineer_features(self):
        # Find Standard Deviation of min, max and avg temp among months
        min_temps = [
            "january_min_temp",
            "february_min_temp",
            "march_min_temp",
            "april_min_temp",
            "may_min_temp",
            "june_min_temp",
            "july_min_temp",
            "august_min_temp",
            "september_min_temp",
            "october_min_temp",
            "november_min_temp",
            "december_min_temp"
        ]

        max_temps = [
            "january_max_temp",
            "february_max_temp",
            "march_max_temp",
            "april_max_temp",
            "may_max_temp",
            "june_max_temp",
            "july_max_temp",
            "august_max_temp",
            "september_max_temp",
            "october_max_temp",
            "november_max_temp",
            "december_max_temp"
        ]

        avg_temps = [
            "january_avg_temp",
            "february_avg_temp",
            "march_avg_temp",
            "april_avg_temp",
            "may_avg_temp",
            "june_avg_temp",
            "july_avg_temp",
            "august_avg_temp",
            "september_avg_temp",
            "october_avg_temp",
            "november_avg_temp",
            "december_avg_temp",
        ]

        self.numeric_features = [
            "floor_area",
            "year_built",
            "energy_star_rating",
            "ELEVATION",
            "cooling_degree_days",
            "heating_degree_days",
            "precipitation_inches",
            "snowfall_inches",
            "snowdepth_inches",
            "avg_temp",
            "days_below_30F",
            "days_below_20F",
            "days_below_10F",
            "days_below_0F",
            "days_above_80F",
            "days_above_90F",
            "days_above_100F",
            "days_above_110F",
            "max_wind_speed",
            "days_with_fog",
            "building_age",
            "min_temp_std",
            "max_temp_std",
            "avg_temp_std",
            "0-10",
            "10-20",
            "20-30",
            "30-80",
            "80-90",
            "90-100",
            "100-110",
            # "site_eui_mean",
            # "site_eui_var"
            
        ] + min_temps + max_temps + avg_temps

        self.categorical_features = [
            "Year_Factor",
            "State_Factor",
            "building_class",
            "facility_type",
            "direction_max_wind_speed",
            "direction_peak_wind_speed"
        ]

        self.drop_columns = [
            "id"
        ]

        for X in [self.X_train, self.X_val, self.X_test]:
            X["building_age"] = 2022 - X["year_built"]

        for X in [self.X_train, self.X_val, self.X_test]:
            X["min_temp_std"] = X[min_temps].T.std()
            X["max_temp_std"] = X[max_temps].T.std()
            X["avg_temp_std"] = X[avg_temps].T.std()

        days_above_below = [
            "days_below_30F",
            "days_below_20F",
            "days_below_10F",
            "days_below_0F",
            "days_above_80F",
            "days_above_90F",
            "days_above_100F",
            "days_above_110F",
        ]

        # Creating windows from days_above and days_below
        # --------------------------------------
        for X in [self.X_train, self.X_val, self.X_test]:
            X["0-10"] = X["days_below_10F"] - X["days_below_0F"]
            X["10-20"] = X["days_below_20F"] - X["days_below_10F"]
            X["20-30"] = X["days_below_30F"] - X["days_below_20F"]
            X["80-90"] = X["days_above_80F"] - X["days_above_90F"]
            X["90-100"] = X["days_above_90F"] - X["days_above_100F"]
            X["100-110"] = X["days_above_100F"] - X["days_above_110F"]
            X["30-80"] = (366 - X[days_above_below].sum(axis=1)).clip(lower=0)
        # --------------------------------------

        # Imputing energy_star_rating
        # --------------------------------------
#         most_freq = defaultdict(int)
#         increment = 5
#         for year in range(1595, 2020, increment):
#             df_temp = self.X_train.loc[
#                 (self.X_train["year_built"] > year) & (self.X_train["year_built"] <= year + increment)
#             ]
#             try:
#                 most_freq[f"{year}-{year + increment}"] = df_temp["energy_star_rating"].value_counts().index[0]
#             except:
#                 most_freq[f"{year}-{year + increment}"] = 0

#         for X in [self.X_train, self.X_val, self.X_test]:
#             for year in range(1595, 2020, increment):
#                 X.loc[
#                     (X["year_built"] > year) & (X["year_built"] <= year + increment),
#                     "energy_star_rating"
#                 ] = X.loc[
#                     (X["year_built"] > year) & (X["year_built"] <= year + increment)
#                 ]["energy_star_rating"].fillna(most_freq[f"{year}-{year + increment}"])
        # --------------------------------------

        # facility_type category seggregation
        # --------------------------------------
        # residential = [
        #     "Multifamily_Uncategorized",
        #     "2to4_Unit_Building",
        #     "5plus_Unit_Building",
        #     "Religious_worship",
        #     "Parking_Garage",
        #     "Mixed_Use_Predominantly_Residential"
        # ]

        # industrial = [
        #     "Warehouse_Nonrefrigerated",
        #     "Warehouse_Distribution_or_Shipping_center",
        #     "Warehouse_Selfstorage",
        #     "Industrial",
        #     "Warehouse_Uncategorized",
        #     "Warehouse_Refrigerated",
        #     "Laboratory",
        #     "Data_Center"
        # ]

        # commercial = list(
        #     set(self.X_train["facility_type"].value_counts().index) -
        #     set(residential) - set(industrial)
        # )

        # f_type_eui = pd.concat([self.X_train["facility_type"], self.y_train], axis=1)[["facility_type", "site_eui"]]
        # eui_means = dict(f_type_eui.groupby("facility_type")["site_eui"].mean().sort_values())
        # eui_var = dict(f_type_eui.groupby("facility_type")["site_eui"].var().sort_values())

        # for X in [self.X_train, self.X_val, self.X_test]:
        # # --------------------------------------
            # for facility_type in [
            #     "Food",
            #     "Education",
            #     "Health_Care",
            #     "Public_Assembly",
            #     "Public_Safety",
            #     "Lodging",
            #     "Warehouse",
            #     "Office",
            #     "Service",
            #     "Retail"
            # ]:
            #     X["facility_type"] = [
            #         facility_type if f_type.lower().startswith(facility_type.lower())
            #         else f_type
            #         for f_type in X["facility_type"]
            #     ]
        # # --------------------------------------
# #             X.replace({"facility_type": convert_dic}, inplace=True)
        # # --------------------------------------
#                 X["site_eui_mean"] = X["facility_type"].map(eui_means)
#                 X["site_eui_var"] = X["facility_type"].map(eui_var)
        # # --------------------------------------
# #         #     X["facility_type"] = [
# #         #             "commercial" if f_type in commercial
# #         #             else "residential" if f_type in residential
# #         #             else "industrial"
# #         #         for f_type in X["facility_type"]]
        # # --------------------------------------


    def preprocess(self):
        knn_imputer = KNNImputer(n_neighbors=5, weights="uniform", metric="nan_euclidean")

        ridge = Ridge(max_iter=10000)
        mice_imputer = IterativeImputer(estimator=ridge)

        pipe_numeric_feats = make_pipeline(
            SimpleImputer(strategy="mean"),
            # knn_imputer,
            # mice_imputer,
            # SimpleImputer(strategy="constant", fill_value=0),
            # SimpleImputer(strategy="most_frequent"),
            StandardScaler()
        )
        pipe_cat_feats = make_pipeline(
            SimpleImputer(strategy="most_frequent"),
            OneHotEncoder(handle_unknown="ignore", sparse=False)
        )
        self.column_transformer = make_column_transformer(
            (pipe_numeric_feats, self.numeric_features),
            (pipe_cat_feats, self.categorical_features)
        )

        self.X_train_raw = self.X_train
        self.X_val_raw = self.X_val
        self.X_test_raw = self.X_test
        self.X_train = self.column_transformer.fit_transform(self.X_train)
        self.X_val = self.column_transformer.transform(self.X_val)
        self.X_test = self.column_transformer.transform(self.X_test)

        self.X_train_tensor = torch.tensor(self.X_train, dtype=torch.float32)
        self.y_train_tensor = torch.tensor(self.y_train.values, dtype=torch.float32)
        self.X_val_tensor = torch.tensor(self.X_val, dtype=torch.float32)
        self.y_val_tensor = torch.tensor(self.y_val.values, dtype=torch.float32)
        self.X_test_tensor = torch.tensor(self.X_test, dtype=torch.float32)

In [360]:
feature_set = Dataprep(train_path, test_path)
feature_set.engineer_features()
feature_set.preprocess()

In [334]:
# pd.read_csv(train_path)["year_built"].sort_values()[-50:]

In [335]:
# feature_set.X_train_raw["energy_star_rating"].isnull().sum()

In [6]:
############################################ Temp zone #################################################

# Apply imputation in all columns

# Tried: X_train["facility_type"].value_counts() -> Categorize in 3-4 categories
# energy level / floor area group by mean and variance

# Time series seasonal component in the monthly data -> Try feeding to RNNs
# Take three month windows to calculate average -> Repeat it to narrow 12 features down to 1 or try SVD

# Tried: use site_eui to order facility_type

# Heating degree days and cooling degree days seem inversely correlated
# ["cooling_degree_days", "heating_degree_days", "precipitation_inches", "site_eui"] <- plot correlation

# Snowfall and snowdepth correlation

# building_class and facility_type seem correlated somehow. Drop one at a time and check

# Check correlation for features at the end of dataset

# Consider dropping days_with_fog

# direction wind speed 360 -> 1

In [7]:
########################################################################################################

In [301]:
def mse(predictions, targets):
    return np.mean((predictions - targets) ** 2)


def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())


def mape(true, pred):
    return 100.0 * np.mean(np.abs((pred - true) / true))


def r2score_torch(predictions, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - predictions) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2


def mse_torch(predictions, targets):
    return torch.mean((predictions - targets) ** 2)


def rmse_torch(predictions, targets):
    return torch.sqrt(((predictions - targets) ** 2).mean())

## Sklearn Models

#### Helper Code

In [302]:
for dt in [
    feature_set.X_train,
    feature_set.X_val,
    feature_set.y_train,
    feature_set.y_val
]:
    print(dt.shape)

(60605, 159)
(15152, 159)
(60605,)
(15152,)


In [303]:
def cross_val_scores(model, X_train, y_train, X_val, y_val, return_train_score=False):

    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    score_dict = {
        "r2_val": model.score(X_val, y_val),
        "mse_val": mse(y_val, y_val_pred),
        "rmse_val": rmse(y_val, y_val_pred),
        "mape_val": mape(y_val, y_val_pred)
    }

    if return_train_score:
        y_train_pred = model.predict(X_train)

        score_dict["r2_train"] = model.score(X_train, y_train)
        score_dict["mse_train"] = mse(y_train, y_train_pred)
        score_dict["rmse_train"] = rmse(y_train, y_train_pred)
        score_dict["mape_train"] = mape(y_train, y_train_pred)

    scores_result = pd.Series(score_dict)

    return model, scores_result

In [ ]:
results = {}

#### Model Definition and Execution

In [361]:
pipe_ridge = make_pipeline(feature_set.column_transformer, Ridge(max_iter=10000))
pipe_lasso = make_pipeline(feature_set.column_transformer, Lasso())
pipe_rf = make_pipeline(feature_set.column_transformer, RandomForestRegressor())
pipe_xgb_tuned = make_pipeline(feature_set.column_transformer, XGBRegressor(
    verbosity=0, eta=0.3, max_depth=7, n_estimators=1000
)) # Final
pipe_xgb = make_pipeline(feature_set.column_transformer, XGBRegressor(verbosity=0))
pipe_lgbm = make_pipeline(feature_set.column_transformer, LGBMRegressor(n_estimators=1000, max_depth=20))
pipe_catboost = make_pipeline(feature_set.column_transformer, CatBoostRegressor(
    verbose=False, learning_rate=0.3, l2_leaf_reg=1, iterations=150, depth=8
))

In [344]:
def train(models, results):
    for name, model in models.items():
        print(f"Start {name}!")
        start_time = time.time()
        _, results[name] = cross_val_scores(
            model,
            feature_set.X_train_raw,
            feature_set.y_train,
            feature_set.X_val_raw,
            feature_set.y_val,
            return_train_score=True
        )

        print(f"Done {name} in {round(time.time() - start_time)} secs!")

    return results


mape_scorer = make_scorer(mape, greater_is_better=False)

scoring_metrics = {
    "neg RMSE": "neg_root_mean_squared_error",
    "r2": "r2",
    "mape": mape_scorer
}

In [345]:
models = {
    "Ridge": pipe_ridge,
    "Lasso": pipe_lasso,
    "Random Forest": pipe_rf,
    "XGB": pipe_xgb,
    "XGB Tuned": pipe_xgb_tuned,
    "LGBM": pipe_lgbm,
    "Cat Boost": pipe_catboost,
}

In [346]:
results = train(models, results)

Start Ridge!
Done Ridge in 1 secs!
Start Lasso!
Done Lasso in 2 secs!
Start Random Forest!
Done Random Forest in 204 secs!
Start XGB!
Done XGB in 87 secs!
Start LGBM!
Done LGBM in 9 secs!
Start Cat Boost!
Done Cat Boost in 23 secs!


#### Model Results

In [ ]:
pd.DataFrame(results)

In [ ]:
pd.DataFrame(results)

In [ ]:
pd.DataFrame(results)

In [ ]:
pd.DataFrame(results)

#### Old Model Results

In [347]:
pd.DataFrame(results) # XGB, LGBM Tuned

,Ridge,Lasso,Random Forest,XGB,LGBM,Cat Boost
r2_val,0.354341,0.187559,0.527537,0.592009,0.494852,0.501856
mse_val,2097.641649,2639.489217,1534.956123,1325.496087,1641.144323,1618.391056
rmse_val,45.800018,51.375960,39.178516,36.407363,40.511040,40.229231
mape_val,53.296557,66.490163,39.891001,36.276421,43.051978,44.505828
r2_train,0.359710,0.176455,0.934033,0.961240,0.725267,0.637857
mse_train,2196.094680,2824.632922,226.255735,132.941729,942.292235,1242.094236
rmse_train,46.862508,53.147276,15.041800,11.530036,30.696779,35.243357
mape_train,59.729823,72.278434,16.353760,16.656916,39.065854,45.821845


In [340]:
pd.DataFrame(results) # Base

,Ridge,Lasso,Random Forest,XGB,LGBM,Cat Boost
r2_val,0.354341,0.187559,0.524554,0.499416,0.454600,0.501856
mse_val,2097.641649,2639.489217,1544.648599,1626.318056,1771.915383,1618.391056
rmse_val,45.800018,51.375960,39.302018,40.327634,42.094125,40.229231
mape_val,53.296557,66.490163,40.069138,44.246587,46.779518,44.505828
r2_train,0.359710,0.176455,0.933694,0.679476,0.544895,0.637857
mse_train,2196.094680,2824.632922,227.419803,1099.346734,1560.939105,1242.094236
rmse_train,46.862508,53.147276,15.080444,33.156398,39.508722,35.243357
mape_train,59.729823,72.278434,16.278015,44.144521,49.533682,45.821845


In [331]:
pd.DataFrame(results) # Base Without days_with_fog

,Ridge,Lasso,Random Forest,XGB,LGBM,Cat Boost
r2_val,0.354345,0.187418,0.537029,0.512321,0.455405,0.499519
mse_val,2097.628159,2639.946800,1504.116653,1584.389737,1769.302643,1625.982653
rmse_val,45.799871,51.380413,38.782943,39.804393,42.063079,40.323475
mape_val,53.303512,66.496578,39.845563,43.903376,46.836678,44.612192
r2_train,0.359678,0.176380,0.934296,0.677562,0.546558,0.637447
mse_train,2196.205824,2824.890699,225.354741,1105.911588,1555.237073,1243.500977
rmse_train,46.863694,53.149701,15.011820,33.255249,39.436494,35.263309
mape_train,59.731748,72.281291,16.327109,43.762745,49.398979,45.822039


In [308]:
pd.DataFrame(results) # Without manual imputation of energy_star_rating

,Ridge,Lasso,Random Forest,XGB,LGBM,Cat Boost
r2_val,0.355553,0.176059,0.475216,0.489144,0.449953,0.482921
mse_val,2093.703762,2676.852644,1704.939061,1659.687812,1787.013863,1679.907833
rmse_val,45.757008,51.738309,41.290908,40.739266,42.273087,40.986679
mape_val,52.944059,67.890788,43.148278,44.440831,46.497741,44.828733
r2_train,0.359788,0.166080,0.927840,0.678022,0.551012,0.639734
mse_train,2195.826736,2860.215925,247.496580,1104.336246,1539.960551,1235.657863
rmse_train,46.859649,53.480987,15.732024,33.231555,39.242331,35.151925
mape_train,59.593558,74.009819,17.512207,43.990462,49.123172,45.772677


In [296]:
pd.DataFrame(results) # Manual imputation of energy_star_rating

,Ridge,Lasso,Random Forest,XGB,LGBM,Cat Boost
r2_val,0.311866,0.121490,0.465312,0.487239,0.428682,0.469744
mse_val,2235.635030,2854.137354,1737.114299,1665.878633,1856.121526,1722.714616
rmse_val,47.282502,53.424127,41.678703,40.815176,43.082729,41.505597
mape_val,55.427400,71.842972,45.312954,48.884131,52.304504,49.872915
r2_train,0.318020,0.111582,0.925979,0.656409,0.519099,0.611797
mse_train,2339.085706,3047.137856,253.880011,1178.465023,1649.415482,1331.475542
rmse_train,48.364095,55.200886,15.933613,34.328778,40.612996,36.489389
mape_train,62.719663,78.632545,19.164886,49.896667,56.368283,52.179141


In [284]:
pd.DataFrame(results) # Manual imputation of energy_star_rating without energy_star_rating

,Ridge,Lasso,Random Forest,XGB,LGBM,Cat Boost
r2_val,0.231617,0.047614,0.344738,0.336390,0.294980,0.336316
mse_val,2496.354024,3094.149415,2128.840877,2155.961814,2290.497318,2156.203635
rmse_val,49.963527,55.625079,46.139364,46.432336,47.859140,46.434940
mape_val,64.079061,79.896285,55.291268,58.787086,61.017798,59.635672
r2_train,0.247316,0.044654,0.906964,0.536207,0.395113,0.490414
mse_train,2581.588586,3276.688840,319.098515,1590.737792,2074.670595,1747.802901
rmse_train,50.809336,57.242369,17.863329,39.884054,45.548552,41.806733
mape_train,71.677074,87.511566,23.056019,59.303700,65.409909,61.970243


In [234]:
pd.DataFrame(results) # Manual imputation of energy_star_rating without cat features

,Ridge,Lasso,Random Forest,XGB,LGBM,Cat Boost
r2_val,0.131385,0.115856,0.220049,0.293731,0.254294,0.261765
mse_val,2821.990798,2872.442741,2533.935719,2294.553019,2422.679588,2398.406889
rmse_val,53.122413,53.595175,50.338213,47.901493,49.220723,48.973533
mape_val,67.024925,70.869938,65.185874,65.189011,66.301310,65.651319
r2_train,0.121480,0.106951,0.890152,0.512620,0.331603,0.399746
mse_train,3013.187046,3063.021198,376.760195,1671.638140,2292.498773,2058.779750
rmse_train,54.892504,55.344568,19.410312,40.885672,47.880046,45.373778
mape_train,73.789729,77.369564,26.517419,65.067741,71.199192,69.225269


In [101]:
pd.DataFrame(results) # With most_frequent imputation

,Ridge,Lasso,Random Forest,XGB,LGBM,Cat Boost
r2_val,0.348260,0.146588,0.511744,0.520589,0.472639,0.512448
mse_val,2023.546352,2649.702684,1515.953301,1488.491144,1637.368432,1513.767977
rmse_val,44.983845,51.475263,38.935245,38.580969,40.464409,38.907171
mape_val,52.360265,68.532643,41.102396,46.277515,49.982083,47.379164
r2_train,0.327581,0.118278,0.933902,0.661349,0.523326,0.614742
mse_train,2303.506448,3020.516924,226.432543,1160.118419,1632.942323,1319.779349
rmse_train,47.994859,54.959230,15.047676,34.060511,40.409681,36.328768
mape_train,60.462206,76.188429,17.900999,49.407101,55.685753,52.309040


In [93]:
pd.DataFrame(results) # With 0 imputation

,Ridge,Lasso,Random Forest,XGB,LGBM,Cat Boost
r2_val,0.326453,0.145838,0.550258,0.528508,0.502220,0.529610
mse_val,2091.253486,2652.031556,1396.374188,1463.904982,1545.525546,1460.482018
rmse_val,45.730225,51.497879,37.368090,38.261011,39.313173,38.216253
mape_val,57.692294,72.422039,38.209667,42.229025,45.167833,41.901280
r2_train,0.307481,0.125044,0.936258,0.674414,0.546649,0.632055
mse_train,2372.365243,2997.340203,218.362125,1115.361823,1553.045019,1260.468940
rmse_train,48.706932,54.747970,14.777081,33.397033,39.408692,35.503084
mape_train,65.572713,81.621806,16.117626,43.255014,49.177577,45.133338


In [85]:
pd.DataFrame(results) # With MICE Imputation

,Ridge,Lasso,Random Forest,XGB,LGBM,Cat Boost
r2_val,0.386025,0.200834,0.522715,0.524589,0.484086,0.529209
mse_val,1906.289985,2481.278388,1481.890584,1476.071217,1601.827957,1461.727851
rmse_val,43.661081,49.812432,38.495332,38.419672,40.022843,38.232550
mape_val,51.287064,65.575628,40.645257,42.440007,44.817064,42.756067
r2_train,0.356733,0.165995,0.929526,0.677539,0.541854,0.636683
mse_train,2203.642552,2857.054498,241.422455,1104.655566,1569.473616,1244.614576
rmse_train,46.942971,53.451422,15.537775,33.236359,39.616583,35.279095
mape_train,59.115682,73.370929,17.103033,43.549156,49.471307,45.588249


In [15]:
feature_set.X_train.shape

(68181, 160)

#### Final Model Results

In [352]:
# test_split = 0.01

# pipe_xgb_tuned = make_pipeline(feature_set.column_transformer, XGBRegressor(verbosity=0, eta=0.3, max_depth=7, n_estimators=1000))

# xgb_model, results["XGB"] = cross_val_scores(
#     pipe_xgb_tuned,
#     feature_set.X_train_raw,
#     feature_set.y_train,
#     feature_set.X_val_raw,
#     feature_set.y_val,
#     return_train_score=True
# )

In [353]:
# pd.DataFrame(results) # Just XGB Updated

,Ridge,Lasso,Random Forest,XGB,LGBM,Cat Boost
r2_val,0.354341,0.187559,0.527537,0.641453,0.494852,0.501856
mse_val,2097.641649,2639.489217,1534.956123,951.486878,1641.144323,1618.391056
rmse_val,45.800018,51.375960,39.178516,30.846181,40.511040,40.229231
mape_val,53.296557,66.490163,39.891001,28.243729,43.051978,44.505828
r2_train,0.359710,0.176455,0.934033,0.954009,0.725267,0.637857
mse_train,2196.094680,2824.632922,226.255735,156.418493,942.292235,1242.094236
rmse_train,46.862508,53.147276,15.041800,12.506738,30.696779,35.243357
mape_train,59.729823,72.278434,16.353760,18.035007,39.065854,45.821845


In [354]:
# preds = xgb_model.predict(feature_set.X_test_raw)

In [356]:
# results_dict = {"id": feature_set.X_test_raw["id"],
#                "site_eui": preds}
# pd.DataFrame(results_dict).set_index("id").to_csv("submission.csv")

#### Feature elimination and Poly features

In [52]:
rfecv = RFECV(Ridge(), min_features_to_select=120, n_jobs=-1)

pipe_xgb_rfecv = make_pipeline(
    feature_set.column_transformer, rfecv, XGBRegressor(verbosity=0)
)

In [53]:
poly_feats = PolynomialFeatures(degree=2)

pipe_poly_ridge = make_pipeline(
    feature_set.column_transformer, rfecv, poly_feats, Ridge()
)

In [54]:
models_rfe_poly_ridge = {
    "XGB rfecv": pipe_xgb_rfecv,
    # "Poly Ridge": pipe_poly_ridge
}

In [55]:
results = train(models_rfe_poly_ridge, results)

Start XGB rfecv!
Done XGB rfecv in 22 secs!


In [56]:
pd.DataFrame(results)

,XGB rfecv,Poly Ridge
r2_val,0.514630,0.381328
mse_val,1506.993403,1920.875815
rmse_val,38.820013,43.827797
mape_val,44.149664,48.838148
r2_train,0.631292,0.415892
mse_train,1263.082954,2000.981302
rmse_train,35.539878,44.732329
mape_train,45.894592,55.636894


#### Hyperparam Tuning

##### Random Forest

In [317]:
# Hyperparam Tune Random Forest

params_rf = {
    'randomforestregressor__n_estimators': [300, 500],
    'randomforestregressor__max_depth': [15, 20, 25, 30],
    'randomforestregressor__max_features': ['auto', 'sqrt']#,
    # 'randomforestregressor__min_samples_split': [10]
}

In [318]:
random_search_rf = RandomizedSearchCV(
    pipe_rf,
    params_rf,
    n_jobs=-1,
    n_iter=20,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [319]:
random_search_rf.fit(feature_set.X_train_raw, feature_set.y_train)

C:\Users\UTKARSH\miniconda3\envs\env_sup\lib\site-packages\sklearn\model_selection\_search.py:289: UserWarning: The total space of parameters 16 is smaller than n_iter=20. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('iterativeimputer',
                                                                                                IterativeImputer(estimator=Ridge(max_iter=10000))),
                                                                                               ('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['floor_area',
                                                                                'year_built',
                                                                                'energy_star_rating',
                                                       

In [320]:
# pd.DataFrame(random_search_rf.cv_results_).sort_values(by="mean_test_r2", ascending=False)
pd.DataFrame(random_search_rf.cv_results_)[[
        # "mean_fit_time",
        # "mean_score_time",
        "params",
        "mean_train_neg RMSE",
        "mean_test_neg RMSE",
        "mean_test_mape",
        "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by="mean_test_r2", ascending=False)

,params,mean_train_neg RMSE,mean_test_neg RMSE,mean_test_mape,mean_train_mape,mean_test_r2,mean_train_r2
13,"{'randomforestregressor__n_estimators': 500, '...",-22.236786,-42.509187,-48.266335,-27.699725,0.472713,0.855823
12,"{'randomforestregressor__n_estimators': 300, '...",-22.320138,-42.516567,-48.212083,-27.813465,0.472519,0.854739
9,"{'randomforestregressor__n_estimators': 500, '...",-26.908325,-42.634606,-48.773381,-34.239787,0.469566,0.788880
8,"{'randomforestregressor__n_estimators': 300, '...",-26.857531,-42.640166,-48.720348,-34.181175,0.469461,0.789681
5,"{'randomforestregressor__n_estimators': 500, '...",-31.741441,-42.969853,-49.916158,-41.145564,0.461039,0.706217
4,"{'randomforestregressor__n_estimators': 300, '...",-31.781662,-43.001163,-49.814130,-41.110949,0.460281,0.705474
1,"{'randomforestregressor__n_estimators': 500, '...",-36.118313,-43.723496,-52.656963,-48.470996,0.441945,0.619615
0,"{'randomforestregressor__n_estimators': 300, '...",-36.150339,-43.767467,-52.706204,-48.547531,0.440805,0.618940
15,"{'randomforestregressor__n_estimators': 500, '...",-22.563287,-45.082965,-55.981445,-31.521882,0.407046,0.851552
14,"{'randomforestregressor__n_estimators': 300, '...",-22.663448,-45.121421,-55.950910,-31.620025,0.406036,0.850225


In [314]:
print(random_search_rf.best_params_)
print(random_search_rf.best_score_)
# {'randomforestregressor__n_estimators': 500, 'randomforestregressor__min_samples_split': 10, 'randomforestregressor__max_features': 'auto', 'randomforestregressor__max_depth': 15}
# 0.4428420765573261

{'randomforestregressor__n_estimators': 500, 'randomforestregressor__min_samples_split': 10, 'randomforestregressor__max_features': 'auto', 'randomforestregressor__max_depth': 15}
0.4428420765573261


##### LGBM

In [ ]:
params_lgbm = {
    'lgbmregressor__n_estimators': [10, 100, 1000],
    'lgbmregressor__max_depth': [5, 10, 15]
}

In [ ]:
random_search_lgbm = RandomizedSearchCV(
    pipe_lgbm,
    params_lgbm,
    n_jobs=-1,
    n_iter=20,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [ ]:
# random_search_lgbm.fit(feature_set.X_train_raw, feature_set.y_train)

In [ ]:
# pd.DataFrame(random_search_lgbm.cv_results_)
pd.DataFrame(random_search_lgbm.cv_results_)[[
        # "mean_fit_time",
        # "mean_score_time",
        "params",
        "mean_train_neg RMSE",
        "mean_test_neg RMSE",
        "mean_test_mape",
        "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by="mean_test_r2", ascending=False)

In [ ]:
print(random_search_lgbm.best_params_)
print(random_search_lgbm.best_score_)
# {'lgbmregressor__n_estimators': 1000, 'lgbmregressor__max_depth': 20}
# 0.49699611783887543

##### XGB

In [63]:
# HyperparamTune XGBoost
params_xgb = {
    'xgbregressor__n_estimators': [10, 100, 1000],
    'xgbregressor__max_depth': [3, 5, 7, 12],
    'xgbregressor__eta': [0.01, 0.03, 0.01, 0.3]
}

In [64]:
random_search_xgb = RandomizedSearchCV(
    pipe_xgb,
    params_xgb,
    n_jobs=-1,
    n_iter=20,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [65]:
random_search_xgb.fit(feature_set.X_train_raw, feature_set.y_train)

RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer()),
                                                                                               ('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['floor_area',
                                                                                'year_built',
                                                                                'energy_star_rating',
                                                                                'ELEVATION',

In [67]:
# pd.DataFrame(random_search_xgb.cv_results_)
pd.DataFrame(random_search_xgb.cv_results_)[[
        "mean_fit_time",
        "mean_score_time",
        "params",
        "mean_train_neg RMSE",
        "mean_test_neg RMSE",
        "mean_test_mape",
        "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by="mean_test_r2", ascending=False)

,mean_fit_time,mean_score_time,params,mean_train_neg RMSE,mean_test_neg RMSE,mean_test_mape,mean_train_mape,mean_test_r2,mean_train_r2
13,574.659637,0.307976,"{'xgbregressor__n_estimators': 1000, 'xgbregre...",-10.903566,-38.522429,-40.791748,-15.783641,0.565928,0.965286
1,356.662588,0.292019,"{'xgbregressor__n_estimators': 1000, 'xgbregre...",-19.550576,-38.947062,-43.210219,-28.687540,0.556450,0.888399
2,1175.295331,0.454385,"{'xgbregressor__n_estimators': 1000, 'xgbregre...",-20.642279,-39.257042,-42.864195,-29.014406,0.549508,0.875529
17,563.405348,0.279453,"{'xgbregressor__n_estimators': 1000, 'xgbregre...",-32.168396,-40.598057,-47.114703,-42.491214,0.518292,0.697873
8,49.140569,0.167951,"{'xgbregressor__n_estimators': 100, 'xgbregres...",-30.877795,-40.947238,-47.232692,-41.330082,0.509899,0.721631
4,307.116411,0.296407,"{'xgbregressor__n_estimators': 1000, 'xgbregre...",-36.729475,-42.003204,-49.669794,-46.950941,0.484413,0.606148
0,628.822790,0.373404,"{'xgbregressor__n_estimators': 1000, 'xgbregre...",-36.973773,-42.168410,-50.787486,-48.621630,0.480431,0.600853
10,134.556621,0.192685,"{'xgbregressor__n_estimators': 100, 'xgbregres...",-35.357155,-42.826538,-48.781719,-45.436484,0.463993,0.635034
18,319.171959,0.258908,"{'xgbregressor__n_estimators': 1000, 'xgbregre...",-40.370169,-43.375100,-53.796786,-52.469732,0.450265,0.524188
11,17.446537,0.161568,"{'xgbregressor__n_estimators': 100, 'xgbregres...",-40.689313,-43.545180,-52.544649,-51.480214,0.445782,0.516650


In [69]:
print(random_search_xgb.best_params_)
print(random_search_xgb.best_score_)

{'xgbregressor__n_estimators': 1000, 'xgbregressor__max_depth': 7, 'xgbregressor__eta': 0.3}
0.5659283801380433


##### CatBoost

In [369]:
params_catboost = {
    "catboostregressor__iterations": [100, 150, 200],
    "catboostregressor__learning_rate": [0.03, 0.1, 0.3],
    "catboostregressor__depth": [8, 12, 16, 20, 24],
    "catboostregressor__l2_leaf_reg": [0.2, 0.5, 1, 3]
}

In [370]:
random_search_catboost = RandomizedSearchCV(
    pipe_catboost,
    params_catboost,
    n_jobs=-1,
    n_iter=50,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [371]:
random_search_catboost.fit(feature_set.X_train_raw, feature_set.y_train)

C:\Users\UTKARSH\miniconda3\envs\env_sup\lib\site-packages\sklearn\model_selection\_search.py:925: UserWarning: One or more of the test scores are non-finite: [-43.48201316          nan -41.87307477 -41.52951193 -45.05189409
          nan -43.50276857 -42.32621674          nan          nan
 -43.33635789 -43.90940125 -44.7844798  -42.90076264          nan
 -44.6091344  -43.16322091          nan -46.03323853 -43.0166333
 -41.95382367          nan          nan -42.31239553 -43.17127395
          nan          nan -44.25778094          nan          nan
          nan -43.7427558  -44.26747783 -43.77898841          nan
          nan -43.05794842 -42.21380211 -42.0216047           nan
          nan -41.75020153 -42.0701414           nan -41.98858339
 -42.47560923 -42.06404535 -42.39976013 -42.7512288           nan]
  category=UserWarning
C:\Users\UTKARSH\miniconda3\envs\env_sup\lib\site-packages\sklearn\model_selection\_search.py:925: UserWarning: One or more of the train scores are non-finite

RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer()),
                                                                                               ('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['floor_area',
                                                                                'year_built',
                                                                                'energy_star_rating',
                                                                                'ELEVATION',

In [372]:
pd.DataFrame(random_search_catboost.cv_results_)[
    [
        # "mean_fit_time",
        # "mean_score_time",
        "param_catboostregressor__iterations",
        "param_catboostregressor__learning_rate",
        "param_catboostregressor__depth",
        "param_catboostregressor__l2_leaf_reg",
        "mean_train_neg RMSE",
        "mean_test_neg RMSE",
        # "mean_test_mape",
        # "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by="mean_test_r2", ascending=False).T

,3,41,2,20,44,38,46,42,37,23,...,26,28,29,30,34,35,39,40,43,49
param_catboostregressor__iterations,150,150,150,150,200,100,100,200,100,200,...,150,100,200,100,100,200,150,150,150,200
param_catboostregressor__learning_rate,0.3,0.3,0.3,0.3,0.1,0.3,0.3,0.1,0.3,0.1,...,0.1,0.3,0.1,0.3,0.3,0.03,0.03,0.3,0.1,0.03
param_catboostregressor__depth,8,8,12,12,8,8,8,12,8,12,...,24,20,24,20,24,24,24,24,20,20
param_catboostregressor__l2_leaf_reg,1,0.5,3,1,0.2,1,3,0.2,0.2,1,...,3,0.2,1,3,0.2,0.2,0.5,1,0.2,0.2
mean_train_neg RMSE,-32.6059,-31.7993,-29.3837,-26.3997,-35.9722,-34.9398,-36.5255,-30.5516,-33.841,-32.344,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean_test_neg RMSE,-41.5295,-41.7502,-41.8731,-41.9538,-41.9886,-42.0216,-42.064,-42.0701,-42.2138,-42.3124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean_test_r2,0.496603,0.491242,0.488325,0.486338,0.48532,0.484581,0.483674,0.483451,0.480011,0.47753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean_train_r2,0.689983,0.705141,0.748244,0.79678,0.622698,0.644017,0.610995,0.727825,0.666058,0.694952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [373]:
print(random_search_catboost.best_params_)
print(random_search_catboost.best_score_)
# {'catboostregressor__learning_rate': 0.3, 'catboostregressor__l2_leaf_reg': 1, 'catboostregressor__iterations': 150, 'catboostregressor__depth': 8}

{'catboostregressor__learning_rate': 0.3, 'catboostregressor__l2_leaf_reg': 1, 'catboostregressor__iterations': 150, 'catboostregressor__depth': 8}
0.4966034898215158


#### Stacking

In [57]:
models_selected = {
    "Ridge": pipe_ridge,
    "XGB": pipe_xgb_tuned,
    "LGBM": pipe_lgbm,
    "CatBoost": pipe_catboost
#     "<>_rfecv": pipe_<>_rfecv,
#     "Poly Ridge": pipe_poly_ridge,
}

In [58]:
stacking_model = StackingRegressor(list(models_selected.items()))

In [59]:
name = "Stacking"

print(f"Start {name}!")
start_time = time.time()

_, results[name] = cross_val_scores(
    stacking_model,
    feature_set.X_train_raw,
    feature_set.y_train,
    feature_set.X_val_raw,
    feature_set.y_val,
    return_train_score=True
)

print(f"Done {name} in {round(time.time() - start_time)} secs!")

Start Stacking!
Done Stacking in 166 secs!


In [60]:
pd.DataFrame(results)

,XGB rfecv,Poly Ridge,Stacking
r2_val,0.514630,0.381328,0.546436
mse_val,1506.993403,1920.875815,1408.242515
rmse_val,38.820013,43.827797,37.526557
mape_val,44.149664,48.838148,41.849547
r2_train,0.631292,0.415892,0.667832
mse_train,1263.082954,2000.981302,1137.908182
rmse_train,35.539878,44.732329,33.732895
mape_train,45.894592,55.636894,43.783458


# FCNN

In [119]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [125]:
def linear_block(input_size, output_size):
    return nn.Sequential(
        nn.Linear(input_size, output_size),
        nn.ReLU(),
        nn.Dropout(0.2)
    )

class Extractlastcell(nn.Module):
    def forward(self, x):
        out, _ = x
        return out[-1]


class EnergyRegressor(nn.Module):
    def __init__(self, input_size):
        super(EnergyRegressor, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=2 * input_size, num_layers=2)
        self.tanh = nn.Tanh()
        self.layers = nn.Sequential(
            linear_block(2 * input_size, 3 * input_size),
            linear_block(3 * input_size, 5 * input_size),
            # linear_block(5 * input_size, 10 * input_size),
            # linear_block(10 * input_size, 7 * input_size),
            # linear_block(7 * input_size, 5 * input_size),
            # linear_block(5 * input_size, 3 * input_size),
            linear_block(5 * input_size, input_size),
            nn.Linear(input_size, 700),
            # nn.Linear(1000, 700),
            # nn.Linear(700, 400),
            # nn.Linear(400, 256),
            # nn.Linear(256, 128),
            nn.Linear(700, 64),
            nn.Linear(64, 1)
        )

    def forward(self, X):
        X = X.to(device)
        # X (sequence length, batch size, input size)
        X = X.reshape(1, X.shape[0], X.shape[1])
        X, _ = self.lstm(X)
        X = X[-1]
        X = self.tanh(X)
        X = self.layers(X)
        return X

In [126]:
trainloader = DataLoader(TensorDataset(feature_set.X_train_tensor, feature_set.y_train_tensor), batch_size=32, shuffle=True)
validloader = DataLoader(TensorDataset(feature_set.X_val_tensor, feature_set.y_val_tensor), batch_size=32, shuffle=True)

In [127]:
model = EnergyRegressor(feature_set.X_train_tensor.shape[1])
model.to(device)

EnergyRegressor(
  (lstm): LSTM(1, 2, num_layers=2)
  (tanh): Tanh()
  (layers): Sequential(
    (0): Sequential(
      (0): Linear(in_features=2, out_features=3, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=3, out_features=5, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
    )
    (2): Sequential(
      (0): Linear(in_features=5, out_features=1, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
    )
    (3): Linear(in_features=1, out_features=700, bias=True)
    (4): Linear(in_features=700, out_features=64, bias=True)
    (5): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [128]:
def trainer(model, criterion, optimizer, trainloader, validloader, epochs):
    train_mse = 0
    train_rmse = 0
    train_r2 = 0
    val_mse = 0
    val_rmse = 0
    val_r2 = 0

    for epoch in range(epochs):
        train_batch_mse = []
        train_batch_rmse = []
        train_batch_r2 = []
        val_batch_mse = []
        val_batch_rmse = []
        val_batch_r2 = []

        model.train(True)

        for X, y in trainloader:
            X = X.to(device)
            y = y.to(device)
            y_hat = model(X).flatten()

            optimizer.zero_grad()
            loss = criterion(y_hat, y)
            loss = loss.to(device)
            loss.backward()
            optimizer.step()
            mse_train = mse_torch(y_hat, y)
            rmse_train = rmse_torch(y_hat, y)
            r2_train = r2score_torch(y_hat, y)
            train_batch_mse.append(mse_train)
            train_batch_rmse.append(rmse_train)
            train_batch_r2.append(r2_train)

        train_mse = torch.sum(torch.Tensor(train_batch_mse)) / len(trainloader)
        train_rmse = torch.sum(torch.Tensor(train_batch_rmse)) / len(trainloader)
        train_r2 = torch.sum(torch.Tensor(train_batch_r2)) / len(trainloader)

        model.eval()

        with torch.no_grad():
            for X_valid, y_valid in validloader:
                X_valid = X_valid.to(device)
                y_valid = y_valid.to(device).flatten()
                y_hat_val = model(X_valid)
                mse_val = mse_torch(y_hat_val, y_valid)
                rmse_val = rmse_torch(y_hat_val, y_valid)
                r2_val = r2score_torch(y_hat_val, y_valid)
                val_batch_mse.append(mse_val)
                val_batch_rmse.append(rmse_val)
                val_batch_r2.append(r2_val)
            val_mse = torch.sum(torch.Tensor(val_batch_mse)) / len(validloader)
            val_rmse = torch.sum(torch.Tensor(val_batch_rmse)) / len(validloader)
            val_r2 = torch.sum(torch.Tensor(val_batch_r2)) / len(validloader) 

        print(f"Epoch {epoch + 1}:\tTrain:\tMSE: {round(train_mse.item(), 4)}. RMSE: {round(train_rmse.item(), 4)}, R2: {round(train_r2.item(), 4)}.")
        print(f"\t\tVal:\tMSE: {round(val_mse.item(), 4)}, RMSE: {round(val_rmse.item(), 4)}, R2: {round(val_r2.item(), 4)}.")
        print("-" * 80)
    return model

In [124]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = criterion.to(device)
trained_model = trainer(model, criterion, optimizer, trainloader, validloader, epochs=30)

Epoch 1:	Train:	MSE: 3535.2891. RMSE: 53.82, R2: -0.1005.
		Val:	MSE: 3168.1995, RMSE: 52.202, R2: -34.8266.
--------------------------------------------------------------------------------
Epoch 2:	Train:	MSE: 3455.76. RMSE: 53.0913, R2: -0.0658.
		Val:	MSE: 3317.2327, RMSE: 52.7304, R2: -34.2324.
--------------------------------------------------------------------------------
Epoch 3:	Train:	MSE: 3449.2209. RMSE: 53.1725, R2: -0.0606.
		Val:	MSE: 3103.2505, RMSE: 50.9328, R2: -32.2138.
--------------------------------------------------------------------------------
Epoch 4:	Train:	MSE: 3451.4553. RMSE: 53.238, R2: -0.0612.
		Val:	MSE: 3109.7168, RMSE: 51.2654, R2: -32.5746.
--------------------------------------------------------------------------------
Epoch 5:	Train:	MSE: 3448.7107. RMSE: 53.1562, R2: -0.0573.
		Val:	MSE: 3104.3606, RMSE: 51.1103, R2: -32.6207.
--------------------------------------------------------------------------------
Epoch 6:	Train:	MSE: 3447.8069. RMSE: 53.

In [ ]:
def predict(model, X):
    return model(X.type(torch.float32))

In [ ]:
predictions = predict(trained_model, feature_set.X_test_tensor)
predictions

In [ ]:
results_dict = {"id": feature_set.X_test_raw["id"],
               "site_eui": predictions.cpu().detach().numpy().flatten()}
pd.DataFrame(results_dict).set_index("id").to_csv("submission.csv")